In [1]:
# Imports
from sqlalchemy import create_engine
import urllib.parse as up
import json
import pandas as pd

- **Login to My SQL**

In [2]:
# SQL Credentials
username = 'root'
password = up.quote_plus('OscarRamirez123@')
host = 'localhost'
database = "movies"

# Create the connection string
connection_str = f'mysql+pymysql://{username}:{password}@{host}/{database}'

# Create the engine
engine = create_engine(connection_str)

- **Load Data files**

In [3]:
df_title = pd.read_csv("title_basics-Copy1.csv")
df_title.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118,"Comedy,Fantasy,Romance"
1,tt0043139,movie,Life of a Beijing Policeman,Wo zhe yi bei zi,0,2013.0,NaN,120,"Drama,History"
2,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020.0,NaN,70,Drama
3,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,NaN,122,Drama
4,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005.0,NaN,100,"Comedy,Horror,Sci-Fi"


In [4]:
folder = "Data/"

with open(f"{folder}tmdb_api_results_2000.json", "r") as f:
    tmdb_api_results_2000 = json.load(f)

with open(f"{folder}tmdb_api_results_2001.json", 'r') as a:
    tmdb_api_results_2001 = json.load(a)

- **Create Tables:**

In [5]:
# Create the table for 2000
engine.execute("CREATE TABLE tmdb_data_2000 ("
               "movie_id VARCHAR(20) PRIMARY KEY, "
               "revenue BIGINT, "
               "budget BIGINT, "
               "certification VARCHAR(10)"
               ")")

# Create the table for 2001
engine.execute("CREATE TABLE tmdb_data_2001 ("
               "movie_id VARCHAR(20) PRIMARY KEY, "
               "revenue BIGINT, "
               "budget BIGINT, "
               "certification VARCHAR(10)"
               ")")

# Insert data into the tmdb_data_2000 table
for movie in tmdb_api_results_2000:
    imdb_id = movie.get('imdb_id')
    revenue = movie.get('revenue')
    budget = movie.get('budget')
    certification = movie.get('certification')

    engine.execute("INSERT INTO tmdb_data_2000 (movie_id, revenue, budget, certification) "
                   "VALUES (%s, %s, %s, %s)",
                   (imdb_id, revenue, budget, certification))

- **Add Data to the Tables:**

In [6]:
# Insert data into the tmdb_data_2001 table
for movie in tmdb_api_results_2001:
    imdb_id = movie.get('imdb_id')
    revenue = movie.get('revenue')
    budget = movie.get('budget')
    certification = movie.get('certification')

    engine.execute("INSERT INTO tmdb_data_2001 (movie_id, revenue, budget, certification) "
                   "VALUES (%s, %s, %s, %s)",
                   (imdb_id, revenue, budget, certification))

- **Check Tables:**

In [7]:
#result_2000 = engine.execute("SELECT * FROM tmdb_data_2000")
#for row in result_2000:
#    print(row)

In [8]:
#result_2001 = engine.execute("SELECT * FROM tmdb_data_2001")
#for row in result_2001:
#    print(row)

- **Creating the Genres Table:**

In [9]:
# Import pandas:
import pandas as pd

In [10]:
# Load the title basics data into a DataFrame
df_title_basics = pd.read_csv("title_basics-Copy1.csv")

# Normalize the genres column
df_title_basics['genres'] = df_title_basics['genres'].str.split(',')

In [11]:
# Create the title_genres DataFrame
df_title_genres = df_title_basics.explode('genres')[['tconst', 'genres']]

In [12]:
# Create the genres table
df_genres = pd.DataFrame(df_title_genres['genres'].unique(), columns=['genre_name'])
df_genres['genre_id'] = df_genres.index + 1

In [13]:
# Merge the title_genres DataFrame with the genres table
df_title_genres = df_title_genres.merge(df_genres, left_on='genres', right_on='genre_name')[['tconst', 'genre_id']]

In [14]:
# Insert data into the genres table
df_genres.to_sql('genres', engine, if_exists='replace', index=False)

25

In [15]:
# Insert data into the title_genres table
df_title_genres.to_sql('title_genres', engine, if_exists='replace', index=False)

153249

In [16]:
# Verify the tables
print(pd.read_sql_query('SELECT * FROM genres LIMIT 5', engine))
print(pd.read_sql_query('SELECT * FROM title_genres LIMIT 5', engine))

  genre_name  genre_id
0     Comedy         1
1    Fantasy         2
2    Romance         3
3      Drama         4
4    History         5
      tconst  genre_id
0  tt0035423         1
1  tt0088751         1
2  tt0100275         1
3  tt0108549         1
4  tt0118652         1
